## Standard Imports

In [7]:
import ee
import os
import time
from pathlib import Path
import requests

print('Imported')

Imported


In [6]:

# Initialize Earth Engine
try:
    ee.Initialize(project = 'glacier-probe-model-475519')
except:
    ee.Authenticate()
    ee.Initialize(project = 'glacier-probe-model-475519')

print('Done')

Done


## 1. Hasdeo Forest RGB Downloader

In [14]:
"""
Hasdeo Forest RGB Image Downloader - VIEWABLE VERSION
Downloads RGB satellite imagery for Hasdeo Arand in standard viewable format.
Images are saved as PNG with proper color scaling for visualization.
"""

import ee
import os
import time
from pathlib import Path
import requests
from PIL import Image
import numpy as np
from io import BytesIO

# --- CONFIGURATION ---

PROJECT_ID = 'glacier-probe-model-475519'
OUTPUT_DIR = 'hasdeo_forest_dataset_rgb'
NUM_IMAGES = 50  
SCALE = 30  # 30m resolution for manageable file sizes
MAX_CLOUD_COVER = 30  # Reduced for better quality images

# Hasdeo Forest Area of Interest
# Coordinates: [lon_min, lat_min, lon_max, lat_max]
HASDEO_REGION = {
    'Hasdeo_Arand_Zoomed': [82.75, 22.95, 82.85, 23.05]
}

# Year ranges for sampling
YEAR_RANGES = [
    ('2018-01-01', '2019-01-01'),
    ('2019-01-01', '2020-01-01'),
    ('2020-01-01', '2021-01-01'),
    ('2021-01-01', '2022-01-01'),
    ('2022-01-01', '2023-01-01'),
    ('2023-01-01', '2024-01-01'),
    ('2024-01-01', '2025-01-01'),
]

# --- INITIALIZATION ---

try:
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Earth Engine initialized for project: {PROJECT_ID}")
except Exception as e:
    print(f"⚠ Initialization failed. Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Authentication successful. Earth Engine initialized.")

# --- UTILITY FUNCTIONS ---

def create_output_dirs():
    """Create output directory structure"""
    Path(OUTPUT_DIR).mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/rgb_images").mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/metadata").mkdir(exist_ok=True)
    print(f"✓ Created output directories in {OUTPUT_DIR}/")

def get_satellite_collection(start_date, end_date, roi, max_cloud_cover):
    """Get Sentinel-2 collection with cloud filtering"""
    # Sentinel-2 Surface Reflectance with cloud masking
    def maskS2clouds(image):
        qa = image.select('QA60')
        # Bits 10 and 11 are clouds and cirrus
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11
        mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
            qa.bitwiseAnd(cirrusBitMask).eq(0))
        return image.updateMask(mask)
    
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover)) \
        .map(maskS2clouds)

    s2_count = sentinel2.size().getInfo()
    
    if s2_count > 0:
        return sentinel2, 'sentinel2'
    else:
        return None, None

def download_rgb_image(image, region, filename):
    """Downloads RGB image in viewable PNG format"""
    try:
        # Select RGB bands (B4=Red, B3=Green, B2=Blue)
        rgb = image.select(['B4', 'B3', 'B2'])
        
        # Normalize to 0-255 range for viewing
        # Sentinel-2 values are 0-10000, we'll use 0-3000 as typical range
        rgb_vis = rgb.visualize(
            min=0,
            max=3000,
            bands=['B4', 'B3', 'B2']
        )
        
        # Get thumbnail URL (this returns a viewable image)
        url = rgb_vis.getThumbURL({
            'region': region,
            'dimensions': 1024,  # 1024x1024 pixels
            'format': 'png'
        })
        
        # Download and save as PNG
        response = requests.get(url, timeout=300)
        if response.status_code == 200:
            # Save directly as PNG
            with open(filename, 'wb') as f:
                f.write(response.content)
            return True
        else:
            print(f"✗ HTTP Error {response.status_code}")
            return False
            
    except Exception as e:
        error_msg = str(e)
        print(f"✗ Error: {error_msg[:100]}...")
        return False

def save_metadata(image, filename, region_name):
    """Save image metadata"""
    try:
        props = image.getInfo()['properties']
        metadata_file = filename
        
        with open(metadata_file, 'w') as f:
            f.write(f"Region: {region_name}\n")
            f.write(f"Satellite: SENTINEL-2\n")
            f.write(f"Image Type: RGB (True Color)\n")
            f.write(f"Format: PNG (Viewable)\n")
            
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            f.write(f"Date: {date}\n")
            f.write(f"Cloud Cover: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'N/A')}%\n")
            f.write(f"Product ID: {props.get('PRODUCT_ID', 'N/A')}\n")
            f.write(f"Bands: R=B4 (Red), G=B3 (Green), B=B2 (Blue)\n")
            f.write(f"Resolution: {SCALE}m\n")
            f.write(f"Processing: Cloud-masked, normalized to 0-255\n")
            
    except Exception as e:
        print(f"  ⚠ Metadata warning: {str(e)[:30]}")

# --- MAIN EXECUTION ---

def main_hasdeo():
    """Main download function for Hasdeo Forest"""
    print("=" * 70)
    print("HASDEO FOREST IMAGE DOWNLOADER - RGB VIEWABLE VERSION")
    print(f"Targeting Central Hasdeo Region: {HASDEO_REGION['Hasdeo_Arand_Zoomed']}")
    print("=" * 70)
    
    create_output_dirs()
    
    downloaded_count = 0
    failed_count = 0
    
    print(f"\nTarget: {NUM_IMAGES} images (Max Cloud Cover: {MAX_CLOUD_COVER}%)")
    print(f"Time periods: {len(YEAR_RANGES)}")
    print(f"Resolution: {SCALE}m | Format: PNG (Viewable RGB)\n")
    
    region_name = 'Hasdeo_Arand_Zoomed'
    coords = HASDEO_REGION[region_name]
    roi = ee.Geometry.Rectangle(coords)

    print(f"🌳 Starting Downloads for: {region_name}")
    print("-" * 70)
    
    # Iterate through each year range
    for start_date, end_date in YEAR_RANGES:
        if downloaded_count >= NUM_IMAGES:
            break
        
        # Get the satellite collection for the year
        collection, satellite_type = get_satellite_collection(start_date, end_date, roi, MAX_CLOUD_COVER)
        
        if collection is None:
            print(f"  {start_date[:4]}: 0 images found")
            continue
        
        try:
            count = collection.size().getInfo()
            if count == 0:
                print(f"  {start_date[:4]}: 0 images found")
                continue
                
            print(f"  {start_date[:4]}: {count} images available")
            
            # Limit download per year
            images_to_download = min(10, count, NUM_IMAGES - downloaded_count)
            if images_to_download <= 0:
                break
                
            images = collection.limit(images_to_download).toList(images_to_download)
            
            for i in range(images_to_download):
                if downloaded_count >= NUM_IMAGES:
                    break
                    
                image = ee.Image(images.get(i))
                
                # Get date
                date_acquired = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
                
                # Filename construction (PNG format)
                filename = f"{OUTPUT_DIR}/rgb_images/{region_name}_{date_acquired}_RGB.png"
                metadata_filename = f"{OUTPUT_DIR}/metadata/{region_name}_{date_acquired}_metadata.txt"
                
                # Skip if exists
                if os.path.exists(filename):
                    downloaded_count += 1
                    print(f"    ⊙ {date_acquired} (exists)")
                    continue
                
                print(f"    ↓ {date_acquired}...", end=' ')
                
                if download_rgb_image(image, roi, filename):
                    save_metadata(image, metadata_filename, region_name)
                    downloaded_count += 1
                    print(f"✓ ({downloaded_count}/{NUM_IMAGES})")
                else:
                    failed_count += 1
                
                time.sleep(2)
                
        except Exception as e:
            print(f"  ✗ Error in collection processing: {str(e)[:50]}...")
            continue
            
    # Summary
    print("\n" + "=" * 70)
    print("DOWNLOAD COMPLETE")
    print("=" * 70)
    print(f"✓ Downloaded: {downloaded_count} images")
    print(f"✗ Failed: {failed_count}")
    print(f"📁 Location: {OUTPUT_DIR}/rgb_images/")
    print("\n✓ Images are in standard PNG format and viewable!")
    print("You can open them with any image viewer or photo app.")
    print("=" * 70)

if __name__ == "__main__":
    main_hasdeo()

✓ Earth Engine initialized for project: glacier-probe-model-475519
HASDEO FOREST IMAGE DOWNLOADER - RGB VIEWABLE VERSION
Targeting Central Hasdeo Region: [82.75, 22.95, 82.85, 23.05]
✓ Created output directories in hasdeo_forest_dataset_rgb/

Target: 50 images (Max Cloud Cover: 30%)
Time periods: 7
Resolution: 30m | Format: PNG (Viewable RGB)

🌳 Starting Downloads for: Hasdeo_Arand_Zoomed
----------------------------------------------------------------------
  2018: 11 images available
    ↓ 2018-01-21... ✓ (1/50)
    ↓ 2018-01-31... ✓ (2/50)
    ↓ 2018-02-15... ✓ (3/50)
    ↓ 2018-03-07... ✓ (4/50)
    ↓ 2018-03-12... ✓ (5/50)
    ↓ 2018-04-21... ✓ (6/50)
    ↓ 2018-11-07... ✓ (7/50)
    ↓ 2018-11-17... ✓ (8/50)
    ↓ 2018-12-07... ✓ (9/50)
    ↓ 2018-12-22... ✓ (10/50)
  2019: 41 images available
    ↓ 2019-01-01... ✓ (11/50)
    ↓ 2019-01-06... ✓ (12/50)
    ↓ 2019-01-11... ✓ (13/50)
    ↓ 2019-01-16... ✓ (14/50)
    ↓ 2019-01-21... ✓ (15/50)
    ↓ 2019-01-31... ✓ (16/50)
    ↓ 2019

## 2. Kangaroo Island Black Summer Bushfire Case

In [16]:
"""
Kangaroo Island Black Summer Bushfire Image Downloader
Downloads RGB satellite imagery for Kangaroo Island, South Australia
Covering the 2019-20 Black Summer bushfire period and recovery
Images are saved as PNG with proper color scaling for visualization.
"""

import ee
import os
import time
from pathlib import Path
import requests

# --- CONFIGURATION ---

PROJECT_ID = 'glacier-probe-model-475519'
OUTPUT_DIR = 'kangaroo_island_black_summer'
NUM_IMAGES = 100  # More images to capture pre-fire, during, and post-fire periods
SCALE = 30  # 30m resolution for manageable file sizes
MAX_CLOUD_COVER = 100  # Allow ALL images including heavy smoke/clouds

# Kangaroo Island Area of Interest - Black Summer Fire Zone
# Coordinates converted from DMS to decimal degrees:
# Latitude: 35°33'41"S to 36°05'12"S = -35.5614 to -36.0867
# Longitude: 136°32'04"E to 138°00'00"E = 136.5344 to 138.0000
KANGAROO_ISLAND_REGION = {
    'Kangaroo_Island_Fire_Zone': [136.5344, -36.0867, 138.0000, -35.5614]
}

# Black Summer bushfires on Kangaroo Island:
# - Started: December 20, 2019 (lightning strikes)
# - Major fires: December 30, 2019 - February 6, 2020
# - Declared safe: February 6, 2020
# Year ranges: 5 years before (2014-2019) and 5 years after (2020-2025)
YEAR_RANGES = [
    # Pre-fire period (5 years before)
    ('2014-01-01', '2014-12-31'),
    ('2015-01-01', '2015-12-31'),
    ('2016-01-01', '2016-12-31'),
    ('2017-01-01', '2017-12-31'),
    ('2018-01-01', '2018-12-31'),
    # Critical fire year
    ('2019-01-01', '2019-12-19'),  # Pre-fire 2019
    ('2019-12-20', '2020-02-06'),  # During fire (Dec 20, 2019 - Feb 6, 2020)
    ('2020-02-07', '2020-12-31'),  # Post-fire recovery 2020
    # Post-fire recovery period (4 more years)
    ('2021-01-01', '2021-12-31'),
    ('2022-01-01', '2022-12-31'),
    ('2023-01-01', '2023-12-31'),
    ('2024-01-01', '2024-12-31'),
]

# --- INITIALIZATION ---

try:
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Earth Engine initialized for project: {PROJECT_ID}")
except Exception as e:
    print(f"⚠ Initialization failed. Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Authentication successful. Earth Engine initialized.")

# --- UTILITY FUNCTIONS ---

def create_output_dirs():
    """Create output directory structure"""
    Path(OUTPUT_DIR).mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/rgb_images").mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/metadata").mkdir(exist_ok=True)
    print(f"✓ Created output directories in {OUTPUT_DIR}/")

def get_satellite_collection(start_date, end_date, roi, max_cloud_cover):
    """Get Sentinel-2 collection WITHOUT cloud masking to preserve smoke/fire effects"""
    # NO CLOUD MASKING - We want to see smoke, clouds, and fire effects!
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))

    s2_count = sentinel2.size().getInfo()
    
    if s2_count > 0:
        return sentinel2, 'sentinel2'
    else:
        return None, None

def download_rgb_image(image, region, filename):
    """Downloads RGB image in viewable PNG format"""
    try:
        # Select RGB bands (B4=Red, B3=Green, B2=Blue)
        rgb = image.select(['B4', 'B3', 'B2'])
        
        # Normalize to 0-255 range for viewing
        # Sentinel-2 values are 0-10000, we'll use 0-3000 as typical range
        rgb_vis = rgb.visualize(
            min=0,
            max=3000,
            bands=['B4', 'B3', 'B2']
        )
        
        # Get thumbnail URL (this returns a viewable image)
        url = rgb_vis.getThumbURL({
            'region': region,
            'dimensions': 2048,  # Higher resolution for large area (2048x2048)
            'format': 'png'
        })
        
        # Download and save as PNG
        response = requests.get(url, timeout=300)
        if response.status_code == 200:
            # Save directly as PNG
            with open(filename, 'wb') as f:
                f.write(response.content)
            return True
        else:
            print(f"✗ HTTP Error {response.status_code}")
            return False
            
    except Exception as e:
        error_msg = str(e)
        print(f"✗ Error: {error_msg[:100]}...")
        return False

def save_metadata(image, filename, region_name, period_label):
    """Save image metadata"""
    try:
        props = image.getInfo()['properties']
        metadata_file = filename
        
        with open(metadata_file, 'w') as f:
            f.write(f"Region: {region_name}\n")
            f.write(f"Location: Kangaroo Island, South Australia\n")
            f.write(f"Event: Black Summer Bushfires (2019-20)\n")
            f.write(f"Period: {period_label}\n")
            f.write(f"Satellite: SENTINEL-2\n")
            f.write(f"Image Type: RGB (True Color)\n")
            f.write(f"Format: PNG (Viewable)\n")
            
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            f.write(f"Date: {date}\n")
            f.write(f"Cloud Cover: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'N/A')}%\n")
            f.write(f"Product ID: {props.get('PRODUCT_ID', 'N/A')}\n")
            f.write(f"Bands: R=B4 (Red), G=B3 (Green), B=B2 (Blue)\n")
            f.write(f"Resolution: {SCALE}m\n")
            f.write(f"Processing: NO cloud masking - smoke and fire effects preserved\n")
            f.write(f"\nContext: Area burned ~211,500 ha (48% of island)\n")
            f.write(f"Fire Period: Dec 20, 2019 - Feb 6, 2020\n")
            
    except Exception as e:
        print(f"  ⚠ Metadata warning: {str(e)[:30]}")

def get_period_label(start_date, end_date):
    """Get descriptive label for the time period"""
    if '2019-12-20' in start_date and '2020-02-06' in end_date:
        return "DURING FIRE (Dec 20, 2019 - Feb 6, 2020)"
    elif int(start_date[:4]) < 2019 or (start_date[:4] == '2019' and '12-19' in start_date):
        return f"PRE-FIRE ({start_date[:4]})"
    elif int(start_date[:4]) >= 2020:
        years_after = int(start_date[:4]) - 2020
        return f"POST-FIRE RECOVERY (+{years_after} years - {start_date[:4]})"
    return start_date[:4]

# --- MAIN EXECUTION ---

def main_kangaroo_island():
    """Main download function for Kangaroo Island Black Summer"""
    print("=" * 80)
    print("KANGAROO ISLAND BLACK SUMMER BUSHFIRE IMAGE DOWNLOADER")
    print(f"Coordinates: {KANGAROO_ISLAND_REGION['Kangaroo_Island_Fire_Zone']}")
    print("Fire Period: December 20, 2019 - February 6, 2020")
    print("Coverage: 5 years pre-fire (2014-2019) + fire period + 5 years post-fire (2020-2024)")
    print("=" * 80)
    
    create_output_dirs()
    
    downloaded_count = 0
    failed_count = 0
    
    print(f"\nTarget: {NUM_IMAGES} images (Max Cloud Cover: {MAX_CLOUD_COVER}%)")
    print(f"Time periods: {len(YEAR_RANGES)}")
    print(f"Resolution: {SCALE}m | Format: PNG (Viewable RGB)\n")
    
    region_name = 'Kangaroo_Island_Fire_Zone'
    coords = KANGAROO_ISLAND_REGION[region_name]
    roi = ee.Geometry.Rectangle(coords)

    print(f"🔥 Starting Downloads for: Kangaroo Island Black Summer")
    print("-" * 80)
    
    # Iterate through each year range
    for start_date, end_date in YEAR_RANGES:
        if downloaded_count >= NUM_IMAGES:
            break
        
        period_label = get_period_label(start_date, end_date)
        
        # Get the satellite collection for the period
        collection, satellite_type = get_satellite_collection(start_date, end_date, roi, MAX_CLOUD_COVER)
        
        if collection is None:
            print(f"  {period_label}: 0 images found")
            continue
        
        try:
            count = collection.size().getInfo()
            if count == 0:
                print(f"  {period_label}: 0 images found")
                continue
                
            print(f"  {period_label}: {count} images available")
            
            # Adjust number of images per period
            # More images during fire period, fewer for other years
            if 'DURING FIRE' in period_label:
                images_per_period = min(20, count, NUM_IMAGES - downloaded_count)
            else:
                images_per_period = min(8, count, NUM_IMAGES - downloaded_count)
                
            if images_per_period <= 0:
                break
                
            images = collection.limit(images_per_period).toList(images_per_period)
            
            for i in range(images_per_period):
                if downloaded_count >= NUM_IMAGES:
                    break
                    
                image = ee.Image(images.get(i))
                
                # Get date
                date_acquired = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
                
                # Filename construction (PNG format)
                period_prefix = period_label.split('(')[0].strip().replace(' ', '_').replace('-', '')
                filename = f"{OUTPUT_DIR}/rgb_images/KI_{date_acquired}_{period_prefix}_RGB.png"
                metadata_filename = f"{OUTPUT_DIR}/metadata/KI_{date_acquired}_{period_prefix}_metadata.txt"
                
                # Skip if exists
                if os.path.exists(filename):
                    downloaded_count += 1
                    print(f"    ⊙ {date_acquired} (exists)")
                    continue
                
                print(f"    ↓ {date_acquired}...", end=' ')
                
                if download_rgb_image(image, roi, filename):
                    save_metadata(image, metadata_filename, region_name, period_label)
                    downloaded_count += 1
                    print(f"✓ ({downloaded_count}/{NUM_IMAGES})")
                else:
                    failed_count += 1
                
                time.sleep(2)
                
        except Exception as e:
            print(f"  ✗ Error in collection processing: {str(e)[:50]}...")
            continue
            
    # Summary
    print("\n" + "=" * 80)
    print("DOWNLOAD COMPLETE - KANGAROO ISLAND BLACK SUMMER DATASET")
    print("=" * 80)
    print(f"✓ Downloaded: {downloaded_count} images")
    print(f"✗ Failed: {failed_count}")
    print(f"📁 Location: {OUTPUT_DIR}/rgb_images/")
    print(f"\n📊 Dataset Coverage:")
    print(f"   • Pre-fire baseline: 2014-2019 (5 years)")
    print(f"   • Active fire period: Dec 20, 2019 - Feb 6, 2020")
    print(f"   • Post-fire recovery: 2020-2024 (5 years)")
    print(f"   • Area burned: ~211,500 hectares (48% of island)")
    print(f"\n✓ Images are in standard PNG format and viewable!")
    print("You can open them with any image viewer or photo app.")
    print("=" * 80)

if __name__ == "__main__":
    main_kangaroo_island()

✓ Earth Engine initialized for project: glacier-probe-model-475519
KANGAROO ISLAND BLACK SUMMER BUSHFIRE IMAGE DOWNLOADER
Coordinates: [136.5344, -36.0867, 138.0, -35.5614]
Fire Period: December 20, 2019 - February 6, 2020
Coverage: 5 years pre-fire (2014-2019) + fire period + 5 years post-fire (2020-2024)
✓ Created output directories in kangaroo_island_black_summer/

Target: 100 images (Max Cloud Cover: 100%)
Time periods: 12
Resolution: 30m | Format: PNG (Viewable RGB)

🔥 Starting Downloads for: Kangaroo Island Black Summer
--------------------------------------------------------------------------------
  PRE-FIRE (2014): 0 images found
  PRE-FIRE (2015): 0 images found
  PRE-FIRE (2016): 0 images found
  PRE-FIRE (2017): 4 images available
    ↓ 2017-03-08... ✓ (1/100)
    ↓ 2017-07-21... ✓ (2/100)
    ⊙ 2017-07-21 (exists)
    ↓ 2017-12-26... ✓ (4/100)
  PRE-FIRE (2018): 28 images available
    ↓ 2018-02-06... ✓ (5/100)
    ⊙ 2018-02-06 (exists)
    ↓ 2018-02-26... ✓ (7/100)
    ⊙ 

## 3. Greater Sydney Case

In [18]:
"""
Greater Sydney Blue Mountains Fringe Satellite Image Downloader
Downloads RGB satellite imagery for the Blue Mountains/Warragamba Dam Catchment Area
Captures: Bushfire (Black Summer 2019-20), Drought, Regrowth, and Deforestation
Images are saved as PNG with proper color scaling for visualization.
"""

import ee
import os
import time
from pathlib import Path
import requests

# --- CONFIGURATION (MODIFIED for Sydney Blue Mountains Fringe) ---

PROJECT_ID = 'glacier-probe-model-475519'
OUTPUT_DIR = 'sydney_blue_mountains_fringe_rgb'
NUM_IMAGES = 75  # 15 images per sub-region
SCALE = 30  # Increased to 30m to ensure successful downloads
MAX_CLOUD_COVER = 100  # Allow all images including smoke/clouds during fires

# Greater Sydney Blue Mountains Fringe - Multiple Sub-Regions
# Multiple smaller regions to ensure visible imagery capture
# Coordinates: [lon_min, lat_min, lon_max, lat_max]
SYDNEY_REGIONS = {
    'Blue_Mts_Katoomba': [150.25, -33.75, 150.35, -33.65],  # Katoomba/Three Sisters area
    'Blue_Mts_Wentworth_Falls': [150.35, -33.75, 150.45, -33.65],  # Wentworth Falls
    'Warragamba_Dam_North': [150.55, -33.95, 150.65, -33.85],  # North of dam
    'Warragamba_Dam_South': [150.55, -34.05, 150.65, -33.95],  # South of dam
    'Penrith_Urban_Edge': [150.65, -33.80, 150.75, -33.70],  # Urban fringe
}

# Date ranges specifically targeting the Black Summer event and recovery/drought periods
YEAR_RANGES = [
    # Pre-fire/drought baseline
    ('2018-01-01', '2019-01-01'), 
    # Peak drought and fire period (Black Summer)
    ('2019-06-01', '2020-06-01'), 
    # Immediate post-fire and start of regrowth
    ('2020-06-01', '2021-06-01'), 
    # Continued recovery and expansion monitoring
    ('2021-06-01', '2022-06-01'),
    ('2022-06-01', '2023-06-01'),
    ('2023-06-01', '2024-06-01'),
    ('2024-06-01', '2025-06-01'),
]

# --- INITIALIZATION ---

try:
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Earth Engine initialized for project: {PROJECT_ID}")
except Exception as e:
    print(f"⚠ Initialization failed. Attempting authentication...")
    ee.Authenticate()
    ee.Initialize(project=PROJECT_ID)
    print(f"✓ Authentication successful. Earth Engine initialized.")

# --- UTILITY FUNCTIONS ---

def create_output_dirs():
    """Create output directory structure"""
    Path(OUTPUT_DIR).mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/rgb_images").mkdir(exist_ok=True)
    Path(f"{OUTPUT_DIR}/metadata").mkdir(exist_ok=True)
    print(f"✓ Created output directories in {OUTPUT_DIR}/")

def get_satellite_collection(start_date, end_date, roi, max_cloud_cover):
    """Get Sentinel-2 collection WITHOUT cloud masking to preserve all effects"""
    sentinel2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(roi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', max_cloud_cover))

    s2_count = sentinel2.size().getInfo()
    
    if s2_count > 0:
        return sentinel2, 'sentinel2'
    else:
        return None, None

def download_rgb_image(image, region, filename):
    """Downloads RGB image in viewable PNG format"""
    try:
        # Select RGB bands (B4=Red, B3=Green, B2=Blue)
        rgb = image.select(['B4', 'B3', 'B2'])
        
        # Normalize to 0-255 range for viewing
        rgb_vis = rgb.visualize(
            min=0,
            max=3000,
            bands=['B4', 'B3', 'B2']
        )
        
        # Get thumbnail URL with adjusted dimensions
        url = rgb_vis.getThumbURL({
            'region': region,
            'dimensions': 512,  # Reduced for better compatibility
            'format': 'png'
        })
        
        # Download and save as PNG
        response = requests.get(url, timeout=300)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
            return True
        else:
            print(f"✗ HTTP Error {response.status_code}")
            return False
            
    except Exception as e:
        error_msg = str(e)
        print(f"✗ Error: {error_msg[:100]}...")
        return False

def save_metadata(image, filename, region_name, period_label):
    """Save image metadata"""
    try:
        props = image.getInfo()['properties']
        metadata_file = filename
        
        with open(metadata_file, 'w') as f:
            f.write(f"Region: {region_name}\n")
            f.write(f"Location: Greater Sydney - Blue Mountains Fringe & Warragamba Catchment\n")
            f.write(f"Research Focus: Bushfire, Drought, Regrowth, Deforestation\n")
            f.write(f"Period: {period_label}\n\n")
            
            f.write(f"Satellite: SENTINEL-2\n")
            f.write(f"Image Type: RGB (True Color)\n")
            f.write(f"Format: PNG (Viewable)\n")
            
            date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
            f.write(f"Date: {date}\n")
            f.write(f"Cloud Cover: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'N/A')}%\n")
            f.write(f"Product ID: {props.get('PRODUCT_ID', 'N/A')}\n")
            f.write(f"Bands: R=B4 (Red), G=B3 (Green), B=B2 (Blue)\n")
            f.write(f"Resolution: {SCALE}m\n")
            f.write(f"Processing: NO cloud masking - smoke and fire effects preserved\n")
            
            f.write(f"\n=== RESEARCH CONTEXT ===\n")
            f.write(f"Black Summer Fires: 2019-2020\n")
            f.write(f"Key Indices for Analysis:\n")
            f.write(f"  • NBR (Normalized Burn Ratio): Fire severity\n")
            f.write(f"  • NDVI (Normalized Difference Vegetation Index): Vegetation health/regrowth\n")
            f.write(f"  • EVI (Enhanced Vegetation Index): Drought impact\n")
            f.write(f"  • NDMI (Normalized Difference Moisture Index): Water stress\n")
            f.write(f"  • NDWI (Normalized Difference Water Index): Drought monitoring\n")
            
    except Exception as e:
        print(f"  ⚠ Metadata warning: {str(e)[:30]}")

def get_period_label(start_date, end_date):
    """Get descriptive label for the time period"""
    if '2018' in start_date:
        return "PRE-FIRE BASELINE (2018)"
    elif '2019-06' in start_date and '2020-06' in end_date:
        return "DROUGHT & BLACK SUMMER FIRES (2019-2020)"
    elif '2020-06' in start_date and '2021' in end_date:
        return "IMMEDIATE POST-FIRE RECOVERY (2020-2021)"
    elif '2021' in start_date:
        return f"POST-FIRE RECOVERY & MONITORING ({start_date[:4]}-{end_date[:4]})"
    elif '2022' in start_date:
        return f"REGROWTH & DEFORESTATION MONITORING ({start_date[:4]}-{end_date[:4]})"
    elif '2023' in start_date or '2024' in start_date:
        return f"LONG-TERM RECOVERY ({start_date[:4]}-{end_date[:4]})"
    return f"{start_date[:4]}-{end_date[:4]}"

# --- MAIN EXECUTION ---

def main_sydney():
    """Main download function for Sydney Blue Mountains Fringe - Multiple Regions"""
    print("=" * 80)
    print("🇦🇺 GREATER SYDNEY BLUE MOUNTAINS FRINGE IMAGE DOWNLOADER")
    print("Multiple Sub-Regions for Complete Coverage")
    print("=" * 80)
    print(f"📍 Regions: {len(SYDNEY_REGIONS)}")
    for name, coords in SYDNEY_REGIONS.items():
        print(f"   • {name}: {coords}")
    print(f"🔥 Event: Black Summer Bushfires (2019-2020)")
    print(f"🌵 Context: Prolonged drought + catastrophic fire + urban expansion")
    print(f"📊 Research: Bushfire, Drought, Regrowth, Deforestation")
    print("=" * 80)
    
    create_output_dirs()
    
    total_downloaded = 0
    total_failed = 0
    
    print(f"\nTarget: {NUM_IMAGES} images total (~{NUM_IMAGES // len(SYDNEY_REGIONS)} per region)")
    print(f"Time periods: {len(YEAR_RANGES)}")
    print(f"Resolution: {SCALE}m | Format: PNG (Viewable RGB)\n")
    
    images_per_region = NUM_IMAGES // len(SYDNEY_REGIONS)
    
    # Process each sub-region
    for region_name, coords in SYDNEY_REGIONS.items():
        print(f"\n{'='*80}")
        print(f"🌳 Processing Region: {region_name}")
        print(f"📍 Coordinates: {coords}")
        print(f"🎯 Target: {images_per_region} images")
        print("-" * 80)
        
        roi = ee.Geometry.Rectangle(coords)
        downloaded_count = 0
        failed_count = 0
        
        # Iterate through each year range for this region
        for start_date, end_date in YEAR_RANGES:
            if downloaded_count >= images_per_region:
                break
            
            period_label = get_period_label(start_date, end_date)
            
            # Get the satellite collection for the period
            collection, satellite_type = get_satellite_collection(start_date, end_date, roi, MAX_CLOUD_COVER)
            
            if collection is None:
                print(f"  {period_label}: 0 images found")
                continue
            
            try:
                count = collection.size().getInfo()
                if count == 0:
                    print(f"  {period_label}: 0 images found")
                    continue
                    
                print(f"  {period_label}: {count} images available")
                
                # Images per period
                if 'BLACK SUMMER' in period_label:
                    images_per_period = min(5, count, images_per_region - downloaded_count)
                else:
                    images_per_period = min(3, count, images_per_region - downloaded_count)
                    
                if images_per_period <= 0:
                    break
                    
                images = collection.limit(images_per_period).toList(images_per_period)
                
                for i in range(images_per_period):
                    if downloaded_count >= images_per_region:
                        break
                        
                    image = ee.Image(images.get(i))
                    
                    # Get date
                    date_acquired = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
                    
                    # Filename construction
                    period_code = period_label.split('(')[0].strip().replace(' ', '_').replace('&', '').replace('-', '')
                    filename = f"{OUTPUT_DIR}/rgb_images/{region_name}_{date_acquired}_{period_code}_RGB.png"
                    metadata_filename = f"{OUTPUT_DIR}/metadata/{region_name}_{date_acquired}_{period_code}_meta.txt"
                    
                    # Skip if exists
                    if os.path.exists(filename):
                        downloaded_count += 1
                        print(f"    ⊙ {date_acquired} (exists)")
                        continue
                    
                    print(f"    ↓ {date_acquired}...", end=' ')
                    
                    if download_rgb_image(image, roi, filename):
                        save_metadata(image, metadata_filename, region_name, period_label)
                        downloaded_count += 1
                        print(f"✓ ({downloaded_count}/{images_per_region})")
                    else:
                        failed_count += 1
                    
                    time.sleep(1.5)
                    
            except Exception as e:
                print(f"  ✗ Error in collection processing: {str(e)[:50]}...")
                continue
        
        total_downloaded += downloaded_count
        total_failed += failed_count
        print(f"\n✓ {region_name}: {downloaded_count} downloaded, {failed_count} failed")
    
    # Final Summary
    print("\n" + "=" * 80)
    print("DOWNLOAD COMPLETE - SYDNEY BLUE MOUNTAINS MULTI-REGION DATASET")
    print("=" * 80)
    print(f"✓ Total Downloaded: {total_downloaded}/{NUM_IMAGES} images")
    print(f"✗ Total Failed: {total_failed}")
    print(f"📁 Location: {OUTPUT_DIR}/rgb_images/")
    print(f"\n📊 Dataset Coverage:")
    print(f"   • Pre-fire baseline: 2018")
    print(f"   • Drought & Black Summer fires: 2019-2020")
    print(f"   • Post-fire recovery: 2020-2021")
    print(f"   • Long-term monitoring: 2021-2025")
    print(f"\n🗺️ Sub-Regions Captured:")
    for name in SYDNEY_REGIONS.keys():
        print(f"   • {name}")
    print(f"\n🔬 Recommended Analysis Indices:")
    print(f"   • NBR (Normalized Burn Ratio): B8-B12 / B8+B12")
    print(f"     → Fire severity mapping, burn scar detection")
    print(f"   • NDVI (Vegetation Index): B8-B4 / B8+B4")
    print(f"     → Vegetation health, regrowth monitoring")
    print(f"   • EVI (Enhanced Vegetation): 2.5*(B8-B4) / (B8+6*B4-7.5*B2+1)")
    print(f"     → Drought stress, canopy density")
    print(f"   • NDMI (Moisture Index): B8-B11 / B8+B11")
    print(f"     → Water stress, drought impact")
    print(f"   • NDWI (Water Index): B3-B8 / B3+B8")
    print(f"     → Drought severity, water availability")
    print(f"\n✓ Images are in standard PNG format and viewable!")
    print("All smoke, fire, and atmospheric effects are preserved.")
    print("=" * 80)

if __name__ == "__main__":
    main_sydney()

✓ Earth Engine initialized for project: glacier-probe-model-475519
🇦🇺 GREATER SYDNEY BLUE MOUNTAINS FRINGE IMAGE DOWNLOADER
Multiple Sub-Regions for Complete Coverage
📍 Regions: 5
   • Blue_Mts_Katoomba: [150.25, -33.75, 150.35, -33.65]
   • Blue_Mts_Wentworth_Falls: [150.35, -33.75, 150.45, -33.65]
   • Warragamba_Dam_North: [150.55, -33.95, 150.65, -33.85]
   • Warragamba_Dam_South: [150.55, -34.05, 150.65, -33.95]
   • Penrith_Urban_Edge: [150.65, -33.8, 150.75, -33.7]
🔥 Event: Black Summer Bushfires (2019-2020)
🌵 Context: Prolonged drought + catastrophic fire + urban expansion
📊 Research: Bushfire, Drought, Regrowth, Deforestation
✓ Created output directories in sydney_blue_mountains_fringe_rgb/

Target: 75 images total (~15 per region)
Time periods: 7
Resolution: 30m | Format: PNG (Viewable RGB)


🌳 Processing Region: Blue_Mts_Katoomba
📍 Coordinates: [150.25, -33.75, 150.35, -33.65]
🎯 Target: 15 images
-------------------------------------------------------------------------------

## Processing 

## Objects:

In [ ]:


image_path = "../sample/sydney_blue_mountains_fringe_rgb/rgb_images/Penrith_Urban_Edge_2019-06-09_DROUGHT__BLACK_SUMMER_FIRES_RGB.png"
process_single_image(image_path, output_dir='sample_processed_output_1')


print(1)
